In [2]:
import warnings
import cv2
import numpy as np
import torch
from einops import repeat, rearrange
from scipy.spatial import distance



warnings.filterwarnings('ignore')

from facexlib import init_detection_model

face_det = init_detection_model('retinaface_mobile0.25', half=True, model_rootpath='../weights')

In [44]:
vid_pth = './video/4ad8f21b54a87b5140bd0c04517be2b0.mp4'
vid_pth_1 = './video/d7c794121f2470e4d766d53555d1d01b.mp4'
# im_pth = '../inputs/tmp/jojo.png'

# frame = cv2.imread(im_pth)
video_stream = cv2.VideoCapture(vid_pth_1)


frames = []
while True:
    ret, frame = video_stream.read()
    if not ret:
        break
    frames.append(frame)

print(len(frames))
print(frames[0])

video_stream.release()


90
[[[ 13  28  51]
  [ 36  51  74]
  [ 48  68  97]
  ...
  [ 67  71  92]
  [ 47  51  72]
  [ 27  31  52]]

 [[ 50  65  88]
  [ 74  89 112]
  [ 91 111 140]
  ...
  [104 108 129]
  [ 82  86 107]
  [ 62  66  87]]

 [[ 49  66  96]
  [ 75  92 122]
  [ 93 118 153]
  ...
  [108 115 135]
  [ 84  88 109]
  [ 62  66  87]]

 ...

 [[ 37  48  63]
  [ 52  63  78]
  [ 66  79  95]
  ...
  [ 87  99 118]
  [ 68  78  95]
  [ 50  60  77]]

 [[ 31  42  57]
  [ 47  58  73]
  [ 58  71  87]
  ...
  [ 80  92 111]
  [ 64  74  91]
  [ 47  57  74]]

 [[  8  19  34]
  [ 23  34  49]
  [ 33  46  62]
  ...
  [ 47  59  78]
  [ 31  41  58]
  [ 15  25  42]]]


In [45]:
def get_center(shape):
    center_x = int(round(np.sum(shape[:, 0]) / 68))
    center_y = int(round(np.sum(shape[:, 1]) / 68)) - 25
    return center_x, center_y

In [55]:
    print('Crop face..')
    idx = 0
    scale_factor = None
    prev_center_x = None
    prev_center_y = None
    prev_roiX1 = None
    prev_roiY1 = None
    prev_roiX2 = None
    prev_roiY2 = None

    frame = frames[0]
    print(frame)
    _frames = repeat(frame,'h w c -> 2 h w c')
    print("------------------------------------------------")
    print(_frames)
    _frames = torch.from_numpy(_frames)
    print('------------------------------------------------')
    print(_frames)
        
    
    

Crop face..
[[[ 13  28  51]
  [ 36  51  74]
  [ 48  68  97]
  ...
  [ 67  71  92]
  [ 47  51  72]
  [ 27  31  52]]

 [[ 50  65  88]
  [ 74  89 112]
  [ 91 111 140]
  ...
  [104 108 129]
  [ 82  86 107]
  [ 62  66  87]]

 [[ 49  66  96]
  [ 75  92 122]
  [ 93 118 153]
  ...
  [108 115 135]
  [ 84  88 109]
  [ 62  66  87]]

 ...

 [[ 37  48  63]
  [ 52  63  78]
  [ 66  79  95]
  ...
  [ 87  99 118]
  [ 68  78  95]
  [ 50  60  77]]

 [[ 31  42  57]
  [ 47  58  73]
  [ 58  71  87]
  ...
  [ 80  92 111]
  [ 64  74  91]
  [ 47  57  74]]

 [[  8  19  34]
  [ 23  34  49]
  [ 33  46  62]
  ...
  [ 47  59  78]
  [ 31  41  58]
  [ 15  25  42]]]
------------------------------------------------
[[[[ 13  28  51]
   [ 36  51  74]
   [ 48  68  97]
   ...
   [ 67  71  92]
   [ 47  51  72]
   [ 27  31  52]]

  [[ 50  65  88]
   [ 74  89 112]
   [ 91 111 140]
   ...
   [104 108 129]
   [ 82  86 107]
   [ 62  66  87]]

  [[ 49  66  96]
   [ 75  92 122]
   [ 93 118 153]
   ...
   [108 115 135]
   [ 84  88 

In [64]:
frames = np.asarray(frames)
print(frames)
print('------------------------------------------------')
frames = torch.from_numpy(frames)
print(frames)

[[[[ 13  28  51]
   [ 36  51  74]
   [ 48  68  97]
   ...
   [ 67  71  92]
   [ 47  51  72]
   [ 27  31  52]]

  [[ 50  65  88]
   [ 74  89 112]
   [ 91 111 140]
   ...
   [104 108 129]
   [ 82  86 107]
   [ 62  66  87]]

  [[ 49  66  96]
   [ 75  92 122]
   [ 93 118 153]
   ...
   [108 115 135]
   [ 84  88 109]
   [ 62  66  87]]

  ...

  [[ 37  48  63]
   [ 52  63  78]
   [ 66  79  95]
   ...
   [ 87  99 118]
   [ 68  78  95]
   [ 50  60  77]]

  [[ 31  42  57]
   [ 47  58  73]
   [ 58  71  87]
   ...
   [ 80  92 111]
   [ 64  74  91]
   [ 47  57  74]]

  [[  8  19  34]
   [ 23  34  49]
   [ 33  46  62]
   ...
   [ 47  59  78]
   [ 31  41  58]
   [ 15  25  42]]]


 [[[ 13  28  51]
   [ 36  51  74]
   [ 48  68  97]
   ...
   [ 67  72  90]
   [ 47  52  70]
   [ 27  32  50]]

  [[ 50  65  88]
   [ 74  89 112]
   [ 91 111 140]
   ...
   [104 109 127]
   [ 82  87 105]
   [ 62  67  85]]

  [[ 49  66  96]
   [ 75  92 122]
   [ 93 118 153]
   ...
   [108 116 133]
   [ 84  89 107]
   [ 62  67

In [72]:
def convert_det_out(bboxes, lmds):
    bbox = bboxes[0, :4]
    lmd = lmds[0]

    landmark = np.array([[lmd[i], lmd[i + 1]] for i in range(0, 10, 2)])

    return bbox, landmark


In [77]:
all_landmarks5 = []
det_faces = []
with torch.no_grad():
    batch_bboxes, batch_lmds = face_det.batched_detect_faces(frames, 0.97)
    print(len(batch_bboxes), len(batch_lmds))
    print(batch_bboxes)
    print('------------------------------------------------')
    print(batch_lmds)
    #for bboxes, lmds in zip(batch_bboxes, batch_lmds):
        #bbox, landmark = convert_det_out(bboxes, lmds)
        #all_landmarks_5.append(landmark)
        #det_faces.append(bbox)
    #print('------------------------------------------------------------------------------------')
    #print(len(det_faces), len(all_landmarks_5))
    #print(det_faces)
    #print('------------------------------------------------')
    #print(all_landmarks_5)
        

90 90
[array([[1.00765625e+03, 2.89114258e+02, 1.09459375e+03, 4.00426758e+02,
        9.99511719e-01],
       [6.73515686e+02, 2.27740234e+02, 7.62703186e+02, 3.37052734e+02,
        9.99023438e-01],
       [2.00982422e+02, 2.28437500e+02, 2.97107422e+02, 3.44812500e+02,
        9.96582031e-01]], dtype=float32), array([[1.0072735e+03, 2.8885303e+02, 1.0942734e+03, 3.9972803e+02,
        9.9951172e-01],
       [6.7278125e+02, 2.2891017e+02, 7.6190619e+02, 3.3716019e+02,
        9.9902344e-01],
       [2.0081738e+02, 2.2853906e+02, 2.9712988e+02, 3.4516406e+02,
        9.9609375e-01]], dtype=float32), array([[6.7152344e+02, 2.2844141e+02, 7.6083594e+02, 3.3744141e+02,
        9.9902344e-01],
       [1.0059512e+03, 2.8824609e+02, 1.0952012e+03, 3.9937109e+02,
        9.9902344e-01],
       [2.0099805e+02, 2.2864064e+02, 2.9718555e+02, 3.4501562e+02,
        9.9658203e-01]], dtype=float32), array([[6.69246094e+02, 2.28343750e+02, 7.58871094e+02, 3.37281250e+02,
        9.99023438e-01],
  

In [ ]:
norm_x = 10
norm_y = 10
idx = 0
while True:
    # Exception handling 1: Detected more than one, find closest to given coordinate.
    _frame = frames[idx]
    print(_frame)
    bbox = batch_bboxes[idx]  
    print('------------------------------------------------')
    print(bbox)
    frame_w, frame_h, frame_c = frames[idx].shape
    if len(bbox) > 1:
        abs_x = round(norm_x * frame_w)
        abs_y = round(norm_y * frame_h)
        closest_rect = None
        closest_distance = 9999
        print('------------------------------------------------')
        for k, rect in enumerate(bbox):
            
            print(rect)
            x1, y1, x2, y2 = rect[0],rect[1],rect[2],rect[3]
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            cur_distance = distance.euclidean((abs_x, abs_y), (center_x, center_y))
            if cur_distance < closest_distance:
                closest_rect = rect
                closest_distance = cur_distance
                batch_bboxes[idx] = [closest_rect]


In [33]:
import argparse
def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--ngpu', help='Number of GPUs across which to run in parallel', default=1, type=int)
    parser.add_argument('--batch_size', help='Single GPU Face detection batch size', default=64, type=int)
    parser.add_argument("--data_root", help="Root folder of the LRS2 dataset", required=True)
    parser.add_argument("--preprocessed_root", help="Root folder of the preprocessed dataset", required=True)

    args = parser.parse_args()

    return args

In [42]:
print(len(frames))
with torch.no_grad():
    args_batch_size  = 1
    for i in range(0, len(frames), args_batch_size):
        fb = np.asarray(frames[i:i + args_batch_size])
        preds = face_det.batched_detect_faces(torch.from_numpy(fb), 0.97)[0]
        print(preds)
        for j, f in enumerate(preds):
            if len(f) == 0:
                continue

90
[array([], dtype=float32)]
[array([], dtype=float32)]
[array([], dtype=float32)]
[array([], dtype=float32)]
[array([], dtype=float32)]
[array([[553.5508   ,  94.40039  , 708.42584  , 314.2754   ,   0.9951172]],
      dtype=float32)]
[array([[554.03125  ,  94.828125 , 708.15625  , 311.57812  ,   0.9941406]],
      dtype=float32)]
[array([[554.7344   ,  94.69531  , 710.1094   , 309.9453   ,   0.9951172]],
      dtype=float32)]
[array([[555.5156    ,  94.70704   , 711.7656    , 306.33203   ,
          0.99365234]], dtype=float32)]
[array([[555.4531   ,  90.796875 , 712.0782   , 305.04688  ,   0.9946289]],
      dtype=float32)]
[array([[554.1953   ,  90.82812  , 709.0704   , 300.82812  ,   0.9941406]],
      dtype=float32)]
[array([[553.9844   ,  86.437515 , 710.2344   , 301.0625   ,   0.9951172]],
      dtype=float32)]
[array([[556.0703   ,  87.97656  , 710.5703   , 303.60156  ,   0.9941406]],
      dtype=float32)]
[array([[558.4297   ,  86.64844  , 712.0547   , 303.52344  ,   0.994140

[array([[553.9004   ,  92.203125 , 703.02545  , 297.45312  ,   0.9941406]],
      dtype=float32)]
[array([[545.7637   ,  85.60938  , 705.5137   , 295.2344   ,   0.9941406]],
      dtype=float32)]
[array([[546.3633   ,  86.09375  , 705.7383   , 296.71875  ,   0.9941406]],
      dtype=float32)]
[array([[546.5371   ,  85.54689  , 705.2872   , 297.54688  ,   0.9941406]],
      dtype=float32)]
[array([[547.4004   ,  86.97656  , 705.9004   , 300.10156  ,   0.9951172]],
      dtype=float32)]
